In [1]:
# Load 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import argparse, time, os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode

import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

# Load 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import argparse, time, os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode


# Define preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to model input size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Define the dataset

import numpy as np
import torch
from torch.utils.data import Subset, DataLoader
from torchvision import datasets, transforms
from sklearn.model_selection import StratifiedShuffleSplit

seed = 42
root = "/home/mrahm326/experiments/Probing Mechanism/figrim/Targets"

# Your existing train transform (from your snippet)
train_transform = transform

# Simple eval transform (adjust to your preprocessing: size/normalize, etc.)
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load once without transforms to read labels for a stratified split
full_base = datasets.ImageFolder(root=root)
y = np.array(full_base.targets)
idx = np.arange(len(y))

# Stratified split (keeps class balance)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
train_idx, test_idx = next(sss.split(idx, y))

# Build separate datasets so train/test can have different transforms
train_dataset = Subset(datasets.ImageFolder(root=root, transform=train_transform), train_idx.tolist())
test_dataset  = Subset(datasets.ImageFolder(root=root, transform=eval_transform),  test_idx.tolist())

# (Optional) DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,  num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import efficientnet_b0
from torchvision import transforms


num_classes = len(full_base.classes)
print(f"Found {num_classes} classes · {len(train_dataset)} train images · {len(test_dataset)} val images")


/home/mrahm326/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Found 21 classes · 508 train images · 128 val images


In [2]:
import torch

# Set device first
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assume adv_patch is a tensor
def create_patch(patch_size=0.2, image_size=224):
    patch_dim = int(patch_size * image_size)
    patch = torch.rand(3, patch_dim, patch_dim, requires_grad=True, device=device)
    return patch
#torch.save(adv_patch, 'adversarial_patch.pth')
adv_patch = create_patch(patch_size=0.2, image_size=224)
import torch

# Load the saved patch
adv_patch = torch.load('/home/mrahm326/experiments/Probing Mechanism/DeepGaze/Figrim_ALL/adversarial_patch_figrim_eff.pth', map_location=device)

# If you need it to require gradients for optimization or testing:
adv_patch.requires_grad = True

# Confirm shape and device
print(f"Loaded patch shape: {adv_patch.shape}, requires_grad: {adv_patch.requires_grad}, device: {adv_patch.device}")

Loaded patch shape: torch.Size([1, 3, 11, 11]), requires_grad: True, device: cuda:0


/tmp/ipykernel_2078397/2256496505.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adv_patch = torch.load('/home/mrahm326/experiments/Probing Mechanism/DeepGaze/Figrim_A

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sal_model = models.segmentation.deeplabv3_resnet50(weights=None, num_classes=1)  # Single channel output

from torchvision import models

# Load pretrained model
sal_model = models.segmentation.deeplabv3_resnet50(weights=models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT)

# Replace the classifier to output a single-channel saliency map
sal_model.classifier[-1] = torch.nn.Conv2d(256, 1, kernel_size=1)
# Load the best trained model
sal_model.load_state_dict(torch.load("/home/mrahm326/experiments/Probing Mechanism/DeepGaze/best_deeplabv3_saliency_cat2000.pt"))
sal_model.eval()

import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pretrained ViT
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim

# Pretrained EfficientNet-B0
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Replace the final fully connected (fc) layer
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

# Move to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# NOTE: This checkpoint is for a ViT and WON'T load into EfficientNet.
# Use an EfficientNet checkpoint trained with the same architecture instead.
# Example (if you have one):

import torch.optim as optim

optimizer = optim.Adam(
    model.parameters(), 
    lr=1e-4,          # learning rate
    betas=(0.9, 0.999),
    weight_decay=1e-4 # L2 regularization
)
criterion = nn.CrossEntropyLoss()

num_epochs = 6
for epoch in range(num_epochs):
    model.train()
    total_loss, total_correct, total_samples = 0.0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += (outputs.argmax(1) == labels).sum().item()
        total_samples += labels.size(0)

    accuracy = 100. * total_correct / total_samples
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")

    # === Validation (optional) ===
    model.eval()
    val_correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_correct += (outputs.argmax(1) == labels).sum().item()
    val_accuracy = 100. * val_correct / len(test_dataset)
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

# === Save Model ===

torch.save(model.state_dict(), "best_res50_figrim.pt")


/tmp/ipykernel_2078397/1668887239.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sal_model.load_state_dict(torch.load("/home/mrahm326/experiments/Probing Mechanism/Dee

'\nimport torch.optim as optim\n\noptimizer = optim.Adam(\n    model.parameters(), \n    lr=1e-4,          # learning rate\n    betas=(0.9, 0.999),\n    weight_decay=1e-4 # L2 regularization\n)\ncriterion = nn.CrossEntropyLoss()\n\nnum_epochs = 6\nfor epoch in range(num_epochs):\n    model.train()\n    total_loss, total_correct, total_samples = 0.0, 0, 0\n\n    for images, labels in train_loader:\n        images, labels = images.to(device), labels.to(device)\n        optimizer.zero_grad()\n\n        outputs = model(images)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n\n        total_loss += loss.item()\n        total_correct += (outputs.argmax(1) == labels).sum().item()\n        total_samples += labels.size(0)\n\n    accuracy = 100. * total_correct / total_samples\n    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")\n\n    # === Validation (optional) ===\n    model.eval()\n    

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torchvision.transforms import InterpolationMode
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
import numpy as np
import random, os
from tqdm import tqdm
import matplotlib.pyplot as plt

# ---------- Config ----------

batch_size = 32
patch_size = 0.05  # size relative to image (e.g., 0.2 → 20% of image height/width)
targeted = True
target_class = 0  # only used in targeted mode
attack_steps = 300
attack_lr = 5e-2

# ---------- Data Transforms ----------

model.to(device)
model.eval()
# ---- helpers (put near your config) ----
mean_t = torch.tensor([0.485, 0.456, 0.406], device=device).view(3,1,1)
std_t  = torch.tensor([0.229, 0.224, 0.225], device=device).view(3,1,1)

def to_model_space(x01):    # [0,1] -> normalized
    return (x01 - mean_t) / std_t

def to_pixel_space(xnorm):  # normalized -> [0,1]
    return torch.clamp(xnorm * std_t + mean_t, 0, 1)

# ---- Patch param & differentiable application ----
def create_patch(patch_size=0.2, image_size=224):
    ph = int(round(patch_size * image_size))
    # Unconstrained parameter; we'll pass it through sigmoid each step
    return nn.Parameter(torch.zeros(3, ph, ph, device=device))

def apply_patch(imgs, patch_norm):
    """
    imgs: normalized images [B,3,H,W]
    patch_norm: normalized patch [3,ph,pw]
    returns: patched images (differentiable w.r.t. patch_norm)
    """
    B, C, H, W = imgs.shape
    ph, pw = patch_norm.shape[1:]
    masks = torch.zeros((B, 1, H, W), device=imgs.device)
    for i in range(B):
        top = random.randint(0, H - ph)
        left = random.randint(0, W - pw)
        masks[i, :, top:top+ph, left:left+pw] = 1.0
    # broadcast patch across spatial dims, mask selects the region
    patched = imgs * (1 - masks) + patch_norm.unsqueeze(0) * masks
    return patched


def apply_patch(imgs, patch_norm):
    """
    imgs: normalized images [B,3,H,W]
    patch_norm: normalized patch [3, ph, pw]
    returns: [B,3,H,W], differentiable w.r.t. patch_norm
    """
    B, C, H, W = imgs.shape
    ph, pw = patch_norm.shape[1:]
    out = []

    for _ in range(B):
        top  = random.randint(0, H - ph)
        left = random.randint(0, W - pw)
        # Pad patch into a full-size canvas at (top,left)
        # F.pad pads (left, right, top, bottom) on the last two dims.
        pad_left, pad_right  = left, W - left - pw
        pad_top,  pad_bottom = top,  H - top  - ph
        patch_canvas = F.pad(patch_norm.unsqueeze(0), (pad_left, pad_right, pad_top, pad_bottom))  # [1,3,H,W]
        patch_canvas = patch_canvas.squeeze(0)  # [3,H,W]

        # Build a 3-channel mask matching the placed region
        mask = torch.zeros((1, H, W), device=imgs.device)
        mask[:, top:top+ph, left:left+pw] = 1.0
        mask3 = mask.expand(C, -1, -1)  # [3,H,W]

        out.append(imgs[0] * (1 - mask3) + patch_canvas)  # blend
        imgs = imgs[1:]  # advance to next image

    return torch.stack(out, dim=0)


def generate_adversarial_patch(model, loader, patch_size=0.2, steps=300, lr=5e-2,
                               targeted=True, target_class=0, image_size=224):
    # parameterize the patch
    patch_param = create_patch(patch_size, image_size)
    optimizer = torch.optim.Adam([patch_param], lr=lr)

    model.eval()
    # (optional) freeze model weights; grads will flow to inputs/patch only
    for p in model.parameters():
        p.requires_grad_(False)

    for step in tqdm(range(steps), desc="Optimizing Patch"):
        total_loss = 0.0
        for imgs, labels in loader:
            imgs = imgs.to(device)

            # param -> [0,1] -> normalized for the model
            patch_01   = torch.sigmoid(patch_param)
            patch_norm = to_model_space(patch_01)

            patched_imgs = apply_patch(imgs, patch_norm)
            outputs = model(patched_imgs)

            if targeted:
                target = torch.full((imgs.size(0),), target_class, dtype=torch.long, device=device)
                loss = F.cross_entropy(outputs, target)
            else:
                loss = -F.cross_entropy(outputs, labels.to(device))

            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if step % 10 == 0:
            print(f"Step {step}/{steps}, Loss: {total_loss:.4f}")

    # return patch in [0,1] space for saving/inspection
    with torch.no_grad():
        return torch.sigmoid(patch_param).detach()

def evaluate_patch_success(model, loader, patch01, targeted=True, target_class=0):
    total = 0
    fooled = 0
    model.eval()
    with torch.no_grad():
        patch_norm = to_model_space(patch01.to(device))
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            patched_imgs = apply_patch(imgs, patch_norm)
            preds = model(patched_imgs).argmax(dim=1)
            fooled += (preds == target_class).sum().item() if targeted else (preds != labels).sum().item()
            total += imgs.size(0)
    print(f'{"Targeted" if targeted else "Untargeted"} Attack Success Rate: {fooled/total:.2%}')

def visualize_patch(patch01):
    # patch01 is in [0,1]
    save_image(patch01, "adv_patch.png")
    print("Adversarial patch saved to adv_patch.png")
    plt.imshow(np.transpose(patch01.detach().cpu().numpy(), (1, 2, 0)))
    plt.title("Adversarial Patch")
    plt.axis("off")
    plt.show()


adv_patch = generate_adversarial_patch(
    model, test_loader, patch_size, attack_steps, attack_lr, targeted, target_class, image_size=224
)
evaluate_patch_success(model, test_loader, adv_patch, targeted, target_class)
visualize_patch(adv_patch)
torch.save(adv_patch, 'adversarial_patch_figrim_res50.pth')

adv_patch = torch.load("adversarial_patch_figrim_res50.pth")

/tmp/ipykernel_2078397/2733081973.py:164: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adv_patch = torch.load("adversarial_patch_figrim_res50.pth")


In [5]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.ndimage import zoom
from scipy.special import logsumexp
import pickle
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import cv2
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

# === Config ===
NUM_CLASSES = num_classes 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# === Hooks ===
def get_hook(storage, name):
    def hook(module, input, output):
        storage[name] = output.detach()
    return hook

block_modules = [(name, module) for name, module in model.named_modules() if isinstance(module, torch.nn.Conv2d)]
layer_names = [name for name, _ in block_modules]

def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union else 0

In [6]:
def apply_patch_random_location(img, adv_patch):
    """
    Apply adversarial patch at a random location for each image in the batch.
    Args:
        img: Tensor of shape [B, C, H, W]
        adv_patch: Tensor of shape [C, ph, pw] or [1, C, ph, pw]
    Returns:
        Tensor with patches applied
    """
    patched = img.clone()
    B, C, H, W = img.shape

    # If adv_patch is batched, squeeze it
    if adv_patch.dim() == 4:
        adv_patch = adv_patch[0]

    _, ph, pw = adv_patch.shape  # now always (C, ph, pw)

    for i in range(B):
        top = torch.randint(0, H - ph + 1, (1,)).item()
        left = torch.randint(0, W - pw + 1, (1,)).item()
        patched[i, :, top:top+ph, left:left+pw] = adv_patch

    return patched


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.ndimage import zoom
from scipy.special import logsumexp
import pickle
from tqdm import tqdm

# === Config ===
NUM_CLASSES = num_classes
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

features_by_class = {i: [] for i in range(NUM_CLASSES)}
labels_by_class = {i: [] for i in range(NUM_CLASSES)}
iou_diffs_by_class = {i: [] for i in range(NUM_CLASSES)}

# === Hooks ===
def get_hook(storage, name):
    def hook(module, input, output):
        storage[name] = output.detach()
    return hook

block_modules = [(name, module) for name, module in model.named_modules() if isinstance(module, torch.nn.Conv2d)]
layer_names = [name for name, _ in block_modules]

def collect_activation_norms(model, img, layer_names):
    activations = {}
    hooks = [dict(model.named_modules())[name].register_forward_hook(get_hook(activations, name)) for name in layer_names]
    _ = model(img)
    for h in hooks: h.remove()
    act_norms = [torch.norm(activations[name]).item() if name in activations else 0.0 for name in layer_names]
    return act_norms

# === Layer Sensitivity Function ===
import torch.nn.functional as F

def compute_sensitivities(model, img, layer_names):
    model.eval()
    activations = {}

    # Get clean output first
    with torch.no_grad():
        clean_output = model(img)
        clean_conf = F.softmax(clean_output, dim=1)

    # Register hooks to capture original activations
    hooks = [dict(model.named_modules())[name].register_forward_hook(get_hook(activations, name)) for name in layer_names]
    _ = model(img)
    for h in hooks:
        h.remove()

    sensitivities = []

    for name in layer_names:
        if name not in activations:
            sensitivities.append(0.0)
            continue

        original_act = activations[name]
        noise = torch.randn_like(original_act) * 0.1
        perturbed_act = original_act + noise

        def override_hook(module, input, output):
            return perturbed_act

        # Register perturbation hook
        temp_hook = dict(model.named_modules())[name].register_forward_hook(override_hook)
        pert_output = model(img)
        pert_conf = F.softmax(pert_output, dim=1)
        temp_hook.remove()

        # Now delta is the effect of perturbing `name`-th layer on the output confidence
        delta = torch.norm(pert_conf - clean_conf).item()
        sensitivities.append(delta)

    #print(sensitivities)
    return sensitivities

@torch.no_grad()
def compute_dataset_sensitivities(model, dataloader, layer_names, device):
    model.eval()
    model.to(device)
    
    total_sensitivities = [0.0 for _ in layer_names]
    count = 0

    for imgs, _ in tqdm(dataloader, desc="Computing sensitivities"):
        imgs = imgs.to(device)
        batch_size = imgs.size(0)

        for i in range(batch_size):
            img = imgs[i].unsqueeze(0)  # single image
            sensitivities = compute_sensitivities(model, img, layer_names)
            total_sensitivities = [total + s for total, s in zip(total_sensitivities, sensitivities)]
            count += 1

    avg_sensitivities = [s / count for s in total_sensitivities]
    return avg_sensitivities

# === Main Loop ===

sens = compute_dataset_sensitivities(model, train_loader, layer_names, DEVICE)
print(sens)

In [10]:
from torch.utils.data import Subset, ConcatDataset
import torch

def get_num_classes_from_loader(loader):
    ds = loader.dataset

    # unwrap Subset(s)
    while isinstance(ds, Subset):
        ds = ds.dataset

    # handle ConcatDataset by merging child datasets' class info
    if isinstance(ds, ConcatDataset):
        class_counts = []
        for child in ds.datasets:
            # unwrap nested Subsets inside the concat
            while isinstance(child, Subset):
                child = child.dataset
            class_counts.append(_num_classes_from_dataset(child, loader=None))
        # assume same label space across children; fall back to max
        return max(class_counts)

    return _num_classes_from_dataset(ds, loader)

def _num_classes_from_dataset(ds, loader=None):
    # Common, fast paths
    if hasattr(ds, "classes"):          # e.g. ImageFolder
        return len(ds.classes)
    if hasattr(ds, "class_to_idx"):     # mapping dict
        return len(ds.class_to_idx)
    if hasattr(ds, "targets"):          # e.g. CIFAR, MNIST
        return len(set(map(int, ds.targets)))
    if hasattr(ds, "labels"):           # some custom datasets
        return len(set(map(int, ds.labels)))

    # Last resort: scan labels from a loader or the dataset itself
    labels_seen = set()
    if loader is not None:
        for _, y in loader:
            y = y if torch.is_tensor(y) else torch.tensor(y)
            labels_seen.update(y.view(-1).tolist())
    else:
        for i in range(len(ds)):
            _, y = ds[i]
            y = y if torch.is_tensor(y) else torch.tensor(y)
            labels_seen.add(int(y.item()))
    return len(labels_seen)

# Usage
num_classes = get_num_classes_from_loader(train_loader)
print("Number of classes:", num_classes)

Number of classes: 21


### Acts

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from scipy.ndimage import zoom
from scipy.special import logsumexp
import pickle

# === Config ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unique_labels = set()
for _, lbl in train_loader:
    unique_labels.update(lbl.tolist())
features_by_class = {int(k): [] for k in sorted(unique_labels)}
labels_by_class   = {int(k): [] for k in sorted(unique_labels)}
iou_diffs_by_class= {int(k): [] for k in sorted(unique_labels)}


all_labels = []
for _, lbl in train_loader:
    all_labels.extend(lbl.tolist())
print("Unique labels:", sorted(set(all_labels))[:30])

for (img, label), (img_raw, _) in zip(train_loader, train_loader):
    img, label = img.to(DEVICE), label.to(DEVICE)
    img.requires_grad = True

    # Gradients (clean)
    gradient_stats_batch = []
    for i in range(img.size(0)):
        model.zero_grad()
        single_img = img[i:i+1].detach()
        single_img.requires_grad = True
        output = model(single_img)
        target_logit = output[0, label[i].item()]
        target_logit.backward()
        gradients = single_img.grad.view(-1).detach().cpu().numpy()
        gradient_stats = [np.mean(gradients), np.std(gradients), np.max(gradients), np.min(gradients)]
        gradient_stats_batch.append(gradient_stats)
        clean_act_norms = collect_activation_norms(model, single_img, layer_names)
        clean_sens = [s * a for s, a in zip(sens, clean_act_norms)]
        features_by_class[label[i].item()].append(clean_sens)
        labels_by_class[label[i].item()].append(0)

    patched = apply_patch_random_location(img, adv_patch)

    gradient_stats_patch_batch = []
    for i in range(patched.size(0)):
        model.zero_grad()
        single_patch = patched[i:i+1].detach()
        single_patch.requires_grad = True
        output_att = model(single_patch)
        target_logit_att = output_att[0, label[i].item()]
        target_logit_att.backward()
        gradients_att = single_patch.grad.view(-1).detach().cpu().numpy()
        grad_stats_att = [np.mean(gradients_att), np.std(gradients_att), np.max(gradients_att), np.min(gradients_att)]
        gradient_stats_patch_batch.append(grad_stats_att)
        patch_act_norms = collect_activation_norms(model, single_patch, layer_names)
        patch_sens = [s * a for s, a in zip(sens, clean_act_norms)]
        features_by_class[label[i].item()].append(patch_sens)
        labels_by_class[label[i].item()].append(1)

# === Train Classifiers ===
from sklearn.metrics import precision_recall_fscore_support
all_precisions, all_recalls, all_f1s = [], [], []


classifiers = {}
for cls in range(21):
    X = np.array(features_by_class[cls])
    y = np.array(labels_by_class[cls])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    clf = RandomForestClassifier(n_estimators=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(f"\nClass {cls} detection report:")
    print(classification_report(y_test, y_pred))

    # Collect per-class metrics for averaging
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average='binary', pos_label=1, zero_division=0
    )
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)

    diffs = np.array(iou_diffs_by_class[cls])
    if len(diffs):
        print(f"IOU change for class {cls}: mean={diffs.mean():.4f}, std={diffs.std():.4f}")

    classifiers[cls] = {
        "classifier": clf,
        "scaler": scaler,
    }

# === Save Classifiers ===
with open("figrim_res50_acts.pkl", "wb") as f:
    pickle.dump(classifiers, f)

print("Saved per-class RF classifiers using separate sensitivities.")
print("\n=== Average Classifier Metrics ===")
print(f"Average Precision: {np.mean(all_precisions):.4f}")
print(f"Average Recall:    {np.mean(all_recalls):.4f}")
print(f"Average F1 Score:  {np.mean(all_f1s):.4f}")

#### Acts and Grads

In [ ]:



def apply_patch_random_location(img, adv_patch):
    """
    Apply adversarial patch at a random location for each image in the batch.
    Args:
        img: Tensor of shape [B, C, H, W]
        adv_patch: Tensor of shape [C, ph, pw]
    Returns:
        patched: Tensor of shape [B, C, H, W] with the patch applied
    """
    patched = img.clone()
    B, C, H, W = img.shape
    ph, pw = adv_patch.shape[1:]

    for i in range(B):
        x = torch.randint(0, H - ph + 1, (1,)).item()
        y = torch.randint(0, W - pw + 1, (1,)).item()
        patched[i, :, x:x+ph, y:y+pw] = adv_patch  # Apply patch to each image

    return patched


unique_labels = set()
for _, lbl in train_loader:
    unique_labels.update(lbl.tolist())
features_by_class = {int(k): [] for k in sorted(unique_labels)}
labels_by_class   = {int(k): [] for k in sorted(unique_labels)}
iou_diffs_by_class= {int(k): [] for k in sorted(unique_labels)}


all_labels = []
for _, lbl in train_loader:
    all_labels.extend(lbl.tolist())
print("Unique labels:", sorted(set(all_labels))[:30])


for (img, label), (img_raw, _) in zip(train_loader, train_loader):
    img, label = img.to(DEVICE), label.to(DEVICE)
    img.requires_grad = True

    # Clean sensitivities

    # Gradients (clean)
    gradient_stats_batch = []
    for i in range(img.size(0)):
        model.zero_grad()
        single_img = img[i:i+1].detach()
        single_img.requires_grad = True
        output = model(single_img)
        target_logit = output[0, label[i].item()]
        target_logit.backward()
        gradients = single_img.grad.view(-1).detach().cpu().numpy()
        gradient_stats = [np.mean(gradients), np.std(gradients), np.max(gradients), np.min(gradients)]
        gradient_stats_batch.append(gradient_stats)
        clean_act_norms = collect_activation_norms(model, single_img, layer_names)
        clean_sens = [s * a for s, a in zip(sens, clean_act_norms)]
        feat_vec = np.concatenate([
            0.85 * np.array(patch_sens, dtype=float),
            0.15  * np.array(grad_stats_att, dtype=float),
        ])

        features_by_class[label[i].item()].append(feat_vec.tolist())
        #features_by_class[label[i].item()].append(clean_sens + gradient_stats)
        labels_by_class[label[i].item()].append(0)


    patched = apply_patch_random_location(img, adv_patch)

    gradient_stats_patch_batch = []
    for i in range(patched.size(0)):
        model.zero_grad()
        single_patch = patched[i:i+1].detach()
        single_patch.requires_grad = True
        output_att = model(single_patch)
        target_logit_att = output_att[0, label[i].item()]
        target_logit_att.backward()
        gradients_att = single_patch.grad.view(-1).detach().cpu().numpy()
        grad_stats_att = [np.mean(gradients_att), np.std(gradients_att), np.max(gradients_att), np.min(gradients_att)]
        gradient_stats_patch_batch.append(grad_stats_att)
        patch_act_norms = collect_activation_norms(model, single_patch, layer_names)
        patch_sens = [s * a for s, a in zip(sens, clean_act_norms)]
        feat_vec = np.concatenate([
            0.85 * np.array(patch_sens, dtype=float),
            0.15  * np.array(grad_stats_att, dtype=float),
            #0.05 * np.array([iou_diff], dtype=float)
        ])

        features_by_class[label[i].item()].append(feat_vec.tolist())
        #features_by_class[label[i].item()].append(patch_sens + grad_stats_att)
        labels_by_class[label[i].item()].append(1)

# === Train Classifiers ===

from sklearn.metrics import precision_recall_fscore_support
all_precisions, all_recalls, all_f1s = [], [], []


classifiers = {}
for cls in range(21):
    X = np.array(features_by_class[cls])
    y = np.array(labels_by_class[cls])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    clf = RandomForestClassifier(n_estimators=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(f"\nClass {cls} detection report:")
    print(classification_report(y_test, y_pred))

    # Collect per-class metrics for averaging
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average='binary', pos_label=1, zero_division=0
    )
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)

    diffs = np.array(iou_diffs_by_class[cls])
    if len(diffs):
        print(f"IOU change for class {cls}: mean={diffs.mean():.4f}, std={diffs.std():.4f}")

    classifiers[cls] = {
        "classifier": clf,
        "scaler": scaler,
    } 

# === Save Classifiers ===
with open("figrim_res50_acts_grads.pkl", "wb") as f:
    pickle.dump(classifiers, f)

print("Saved per-class RF classifiers using separate sensitivities.")
print("\n=== Average Classifier Metrics ===")
print(f"Average Precision: {np.mean(all_precisions):.4f}")
print(f"Average Recall:    {np.mean(all_recalls):.4f}")
print(f"Average F1 Score:  {np.mean(all_f1s):.4f}")

Unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Class 0 detection report:
              precision    recall  f1-score   support

           0       0.55      0.75      0.63         8
           1       0.50      0.29      0.36         7

    accuracy                           0.53        15
   macro avg       0.52      0.52      0.50        15
weighted avg       0.52      0.53      0.51        15


Class 1 detection report:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80         8
           1       1.00      0.43      0.60         7

    accuracy                           0.73        15
   macro avg       0.83      0.71      0.70        15
weighted avg       0.82      0.73      0.71        15


Class 2 detection report:
              precision    recall  f1-score   support

           0       0.64      0.88      0.74         8
           1       0.75      0.43      0.55         7

    accurac

#### Acts, grads and saliency 

In [13]:

unique_labels = set()
for _, lbl in train_loader:
    unique_labels.update(lbl.tolist())
features_by_class = {int(k): [] for k in sorted(unique_labels)}
labels_by_class   = {int(k): [] for k in sorted(unique_labels)}
iou_diffs_by_class= {int(k): [] for k in sorted(unique_labels)}

sal_model = sal_model.to(device)

all_labels = []
for _, lbl in train_loader:
    all_labels.extend(lbl.tolist())
print("Unique labels:", sorted(set(all_labels))[:30])


for (img, label), (img_raw, _) in zip(train_loader, train_loader):
    img, label = img.to(DEVICE), label.to(DEVICE)
    img.requires_grad = True

    # Clean sensitivities
    target_layer = [m for n, m in model.named_modules() if isinstance(m, torch.nn.Conv2d)][-1]
    cam = GradCAM(model=model, target_layers=[target_layer])

    # Gradients (clean)
    gradient_stats_batch = []
    for i in range(img.size(0)):
        model.zero_grad()
        single_img = img[i:i+1].detach()
        single_img.requires_grad = True
        output = model(single_img)
        target_logit = output[0, label[i].item()]
        target_logit.backward()
        gradients = single_img.grad.view(-1).detach().cpu().numpy()
        gradient_stats = [np.mean(gradients), np.std(gradients), np.max(gradients), np.min(gradients)]
        gradient_stats_batch.append(gradient_stats)
        clean_act_norms = collect_activation_norms(model, single_img, layer_names)
        clean_sens = [s * a for s, a in zip(sens, clean_act_norms)]

        if single_img.ndim == 3:
            single_img = single_img.unsqueeze(0) 

        cam_map = cam(input_tensor=single_img, targets=[ClassifierOutputTarget(output.argmax().item()+1)])[0]
        cam_min, cam_max = cam_map.min(), cam_map.max()
        cam_norm = (cam_map - cam_min) / (cam_max - cam_min + 1e-8)
        th1 = np.quantile(cam_norm, 0.80)
        gradcam_mask = (cam_norm >= th1).astype(np.uint8)
        
        sal_out = torch.sigmoid(sal_model(single_img)['out']).squeeze().detach().cpu().numpy()
        sal_min, sal_max = sal_out.min(), sal_out.max()
        sal_norm = (sal_out - sal_min) / (sal_max - sal_min + 1e-8)
        th2 = np.quantile(sal_norm, 0.80)
        human_mask = (sal_norm >= th2).astype(np.uint8)

        if gradcam_mask.shape != human_mask.shape:
            gradcam_mask = cv2.resize(gradcam_mask.astype(np.float32), (human_mask.shape[1], human_mask.shape[0]))
            gradcam_mask = (gradcam_mask > 1).astype(np.uint8)

        iou_diff = 1 - compute_iou(gradcam_mask, human_mask)
        #print(clean_sens, gradient_stats, iou_diff)
        feat_vec =  np.concatenate([
            .20 * np.array(clean_sens, dtype=float),
            .70  * np.array(gradient_stats, dtype=float),
            .05 * np.array([iou_diff], dtype=float)
        ])
        features_by_class[label[i].item()].append(feat_vec.tolist())
        labels_by_class[label[i].item()].append(0)


    patched = apply_patch_random_location(img, adv_patch)

    gradient_stats_patch_batch = []
    for i in range(patched.size(0)):
        model.zero_grad()
        single_patch = patched[i:i+1].detach()
        single_patch.requires_grad = True
        output_att = model(single_patch)
        target_logit_att = output_att[0, label[i].item()]
        target_logit_att.backward()
        gradients_att = single_patch.grad.view(-1).detach().cpu().numpy()
        grad_stats_att = [np.mean(gradients_att), np.std(gradients_att), np.max(gradients_att), np.min(gradients_att)]
        gradient_stats_patch_batch.append(grad_stats_att)
        patch_act_norms = collect_activation_norms(model, single_patch, layer_names)
        patch_sens = [s * a for s, a in zip(sens, clean_act_norms)]


        if single_patch.ndim == 3:
            single_patch = single_patch.unsqueeze(0)

        cam_map = cam(input_tensor=single_patch, targets=[ClassifierOutputTarget(output.argmax().item()+1)])[0]
        cam_min, cam_max = cam_map.min(), cam_map.max()
        cam_norm = (cam_map - cam_min) / (cam_max - cam_min + 1e-8)
        th1 = np.quantile(cam_norm, 0.80)
        gradcam_mask = (cam_norm >= th1).astype(np.uint8)
        
        sal_out = torch.sigmoid(sal_model(single_img)['out']).squeeze().detach().cpu().numpy()
        sal_min, sal_max = sal_out.min(), sal_out.max()
        sal_norm = (sal_out - sal_min) / (sal_max - sal_min + 1e-8)
        th2 = np.quantile(sal_norm, 0.80)
        human_mask = (sal_norm >= th2).astype(np.uint8)

        if gradcam_mask.shape != human_mask.shape:
            gradcam_mask = cv2.resize(gradcam_mask.astype(np.float32), (human_mask.shape[1], human_mask.shape[0]))
            gradcam_mask = (gradcam_mask > 1).astype(np.uint8)
        
        iou_diff = 1 - compute_iou(gradcam_mask, human_mask)

        feat_vec = np.concatenate([
            .20 * np.array(patch_sens, dtype=float),
            .70  * np.array(grad_stats_att, dtype=float),
            .05 * np.array([iou_diff], dtype=float)
        ])

        features_by_class[label[i].item()].append(feat_vec.tolist())
        labels_by_class[label[i].item()].append(1)

# === Train Classifiers ===

from sklearn.metrics import precision_recall_fscore_support
all_precisions, all_recalls, all_f1s = [], [], []

classifiers = {}
for cls in range(21):
    X = np.array(features_by_class[cls])
    y = np.array(labels_by_class[cls])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    clf = RandomForestClassifier(n_estimators=10, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(f"\nClass {cls} detection report:")
    print(classification_report(y_test, y_pred))

    # Collect per-class metrics for averaging
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred, average='binary', pos_label=1, zero_division=0
    )
    all_precisions.append(precision)
    all_recalls.append(recall)
    all_f1s.append(f1)

    diffs = np.array(iou_diffs_by_class[cls])
    if len(diffs):
        print(f"IOU change for class {cls}: mean={diffs.mean():.4f}, std={diffs.std():.4f}")

    classifiers[cls] = {
        "classifier": clf,
        "scaler": scaler,
    }

# === Save Classifiers ===
with open("figrim_res50_acts_grads_iou.pkl", "wb") as f:
    pickle.dump(classifiers, f)

print("Saved per-class RF classifiers using separate sensitivities.")
print("\n=== Average Classifier Metrics ===")
print(f"Average Precision: {np.mean(all_precisions):.4f}")
print(f"Average Recall:    {np.mean(all_recalls):.4f}")
print(f"Average F1 Score:  {np.mean(all_f1s):.4f}")


Unique labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Class 0 detection report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         9

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15


Class 1 detection report:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         8
           1       0.64      1.00      0.78         7

    accuracy                           0.73        15
   macro avg       0.82      0.75      0.72        15
weighted avg       0.83      0.73      0.72        15


Class 2 detection report:
              precision    recall  f1-score   support

           0       0.56      1.00      0.71         5
           1       1.00      0.60      0.75        10

    accurac